<a href="https://colab.research.google.com/github/cytoscape/cytoscape-automation/blob/master/for-scripters/Python/filtering-networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Filtering Networks


## Yihang Xin and Alex Pico

## 2021-11-16


This notebook will introduce you to some techniques for filtering a network based on node properties. You will learn to:

* Select a set of nodes based on node degree and attribute filters
* Create a subnetwork based on selected nodes
* Hide a set of nodes based on filters

For this tutorial, we will use data from the STRING database (https://string-db.org/).

You can also install app inside Python notebook by running "py4cytoscape.install_app('Your App')"

# Installation
The following chunk of code installs the `py4cytoscape` module.

In [1]:
%%capture
!python3 -m pip install python-igraph requests pandas networkx
!python3 -m pip install py4cytoscape

If you are using a remote notebook environment such as Google Colab, please execute the cell below. (If you're running on your local notebook, you don't need to do that.)



In [2]:
import requests
exec(requests.get("https://raw.githubusercontent.com/cytoscape/jupyter-bridge/master/client/p4c_init.py").text)
IPython.display.Javascript(_PY4CYTOSCAPE_BROWSER_CLIENT_JS) # Start browser client

You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


Loading Javascript client ... d6008ed2-69e3-473b-8225-fa394fb7fe29 on https://jupyter-bridge.cytoscape.org


<IPython.core.display.Javascript object>

# Prerequisites
In addition to this package (py4cytoscape version 0.0.11), you will need:

* Latest version of Cytoscape, which can be downloaded from https://cytoscape.org/download.html. Simply follow the installation instructions on screen.

* Complete installation wizard

* Launch Cytoscape

For this vignette, you’ll also need the STRING app:

Install the STRING app from https://apps.cytoscape.org/apps/stringapp

You can also install app inside Python notebook by running py4cytoscape.install_app('Your App')

# Import the required package


In [3]:
import os
import sys
import pandas as pd
import py4cytoscape as p4c

# Setup Cytoscape


In [4]:
p4c.cytoscape_version_info()

{'apiVersion': 'v1',
 'cytoscapeVersion': '3.9.0',
 'automationAPIVersion': '1.2.0',
 'py4cytoscapeVersion': '0.0.10'}

# Get network from STRING


We are going to query the STRING Disease database for the term “breast cancer”. By default, the app pulls the top 100 human proteins associated with the disease along with edges having an evidence strength of 0.4 or greater:

In [5]:
string_cmd = 'string disease query disease="breast cancer"'
p4c.commands.commands_run(string_cmd)

["Loaded network 'STRING network - breast cancer' with 100 nodes and 2500 edges"]

In [6]:
string_net = p4c.get_network_suid()

# Filtering by degree


## Creating a degree filter


Every node in a network has a Degree property, which corresponds to the number of edges connecting the node to other nodes, either as a target or source. Filtering based on node degree is a useful way to remove nodes with too few (or too many) connections.

In this example we want to exclude low degree nodes, e.g., those with only 0, 1 or 2 connections:

In [7]:
p4c.create_degree_filter('degree filter', [0, 2], predicate='IS_NOT_BETWEEN')

No edges selected.


{'nodes': ['9606.ENSP00000261769',
  '9606.ENSP00000407586',
  '9606.ENSP00000405330',
  '9606.ENSP00000451828',
  '9606.ENSP00000372023',
  '9606.ENSP00000269141',
  '9606.ENSP00000418915',
  '9606.ENSP00000263735',
  '9606.ENSP00000371194',
  '9606.ENSP00000441765',
  '9606.ENSP00000361021',
  '9606.ENSP00000256078',
  '9606.ENSP00000484824',
  '9606.ENSP00000352408',
  '9606.ENSP00000437955',
  '9606.ENSP00000275493',
  '9606.ENSP00000229239',
  '9606.ENSP00000410294',
  '9606.ENSP00000351273',
  '9606.ENSP00000343925',
  '9606.ENSP00000344456',
  '9606.ENSP00000368632',
  '9606.ENSP00000261584',
  '9606.ENSP00000277541',
  '9606.ENSP00000263923',
  '9606.ENSP00000278616',
  '9606.ENSP00000336701',
  '9606.ENSP00000227507',
  '9606.ENSP00000268035',
  '9606.ENSP00000267101',
  '9606.ENSP00000252242',
  '9606.ENSP00000356150',
  '9606.ENSP00000295736',
  '9606.ENSP00000418960',
  '9606.ENSP00000219746',
  '9606.ENSP00000324856',
  '9606.ENSP00000265433',
  '9606.ENSP00000479618',
  '

At the bottom of the Select tab, you can see how many edges/nodes where selected.



# Creating a subnetwork from a selection


We can now create a new network, or subnetwork, from our selected set of nodes and all relevant edges:



In [ ]:
p4c.create_subnetwork(subnetwork_name='Breast cancer: highly connected nodes')

# Filtering by attribute


## Creating a column filter


We could also filter the network based on high disease score. The disease score comes from STRING and indicates the strength of the association to the disease queried.

Let’s select nodes from the original network with a disease score of greater than 4 (on a scale of 1-5):

In [ ]:
p4c.create_column_filter('disease score filter','stringdb::disease score', 4, "GREATER_THAN", network = string_net)

Again, we can create a subnetwork from the selection:



In [ ]:
p4c.create_subnetwork(subnetwork_name='Breast cancer: high disease score')

# Combining filters


But what if we want to combine these two filters? You could apply them sequentially as individual filters, but then you’d need to be careful about the order in which you apply the filters. Alternatively, you can create a composite filter and apply the logic all at once!

Let’s combine the two filters “degree filter” and “disease score” to produce one filter, then apply it to the original network and create a final subnetwork:

In [ ]:
p4c.create_composite_filter('combined filter', ['degree filter','disease score filter'], network=string_net)
p4c.create_subnetwork(subnetwork_name='final subnetwork')

We can apply a layout to help with interpreting the network:



In [ ]:
p4c.layout_network('force-directed')

This final network obviously contains fewer nodes than the original, but they are the most connected and most highly associated with the disease. If you examine the network you can see several well-known breast cancer oncogenes, for example BRCA1, TP53 and PTEN, near the center of the action.

# Hiding filtered nodes


As a final example of the filter functions, let’s return to the orignal network once more and apply our “combined filter”. But this time let’s hide the filtered out nodes, rather than forming a selection. This demonstrates the applyFilter function and the hide parameter that is optional for all createXXXFilter functions as well.

In [ ]:
# p4c.apply_filter('combined filter', hide=True, network=string_net)